# Iris Demo - Classic Machine Learning
In this notebook the Iris dataset is used to train a machine learning model. This is just to demonstrate the standard model development process before moving on to a federated learning model. Some of this code was generated by ChatGPT.

In [1]:
# load libraries
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

In [3]:
# Print the feature data
print("Feature data:")
print(iris.data)

# Print the target labels
print("\nTarget labels:")
print(iris.target)

# Print the target class names
print("\nTarget class names:")
print(iris.target_names)

# Print the dataset description
print("\nDataset description:")
print(iris.DESCR)

# Print the feature names
print("\nFeature names:")
print(iris.feature_names)

Feature data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5

In [4]:
# Shuffle the dataset
indices = np.arange(len(X))
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

In [5]:
# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# Split the shuffled data into three equal sets
X_set, y_set = np.split(X, 3), np.split(y, 3)

# Separate features and labels for each set
X_client1, y_client1 = X_set[0], y_set[0]
X_client2, y_client2 = X_set[1], y_set[1]
X_client3, y_client3 = X_set[2], y_set[2]

# Print the sizes of each set
print("Client 1 - Size:", len(X_client1))
print("Client 2 - Size:", len(X_client2))
print("Client 3 - Size:", len(X_client3))

Client 1 - Size: 50
Client 2 - Size: 50
Client 3 - Size: 50


In [7]:
# Split the dataset into training and testing sets
X_client1_train, X_client1_test, y_client1_train, y_client1_test = train_test_split(
    X_client1, y_client1, test_size=0.2, random_state=42
)

# Split the dataset into training and testing sets
X_client2_train, X_client2_test, y_client2_train, y_client2_test = train_test_split(
    X_client2, y_client2, test_size=0.2, random_state=42
)

# Split the dataset into training and testing sets
X_client3_train, X_client3_test, y_client3_train, y_client3_test = train_test_split(
    X_client3, y_client3, test_size=0.2, random_state=42
)

In [8]:
# Create a sequential model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=(4,), activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# NOTE: verbose is set to 0 so there is no feedback, set to 1 if you want feedback
# Train each client model separately
client_models = []

# Client 1 training
model.fit(X_client1_train, y_client1_train, epochs=50, batch_size=8, verbose=0, validation_data=(X_client1_test, y_client1_test))
client_models.append(model)  # Save the trained model for client 1

# Client 2 training
model.fit(X_client2_train, y_client2_train, epochs=50, batch_size=8, verbose=0, validation_data=(X_client2_test, y_client2_test))
client_models.append(model)  # Save the trained model for client 2

# Client 3 training
model.fit(X_client3_train, y_client3_train, epochs=50, batch_size=8, verbose=0, validation_data=(X_client3_test, y_client3_test))
client_models.append(model)  # Save the trained model for client 3

In [10]:
loss, accuracy = client_models[0].evaluate(X_client1_test, y_client1_test, verbose=0)
print(f'Client 1 - Test loss:', loss)
print(f'Client 1 - Test accuracy:', accuracy)

loss, accuracy = client_models[1].evaluate(X_client2_test, y_client2_test, verbose=0)
print(f'\nClient 2 - Test loss:', loss)
print(f'Client 2 - Test accuracy:', accuracy)

loss, accuracy = client_models[2].evaluate(X_client3_test, y_client3_test, verbose=0)
print(f'\nClient 3 - Test loss:', loss)
print(f'Client 3 - Test accuracy:', accuracy)

Client 1 - Test loss: 0.3981778919696808
Client 1 - Test accuracy: 0.8999999761581421

Client 2 - Test loss: 0.2610824704170227
Client 2 - Test accuracy: 1.0

Client 3 - Test loss: 0.3765501081943512
Client 3 - Test accuracy: 0.8999999761581421
